In [ ]:
from pymongo import MongoClient

# Connect to your MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["your_db_name"]
collection = db["your_collection_name"]

# Fetch your scraped documents (assume stored under 'content' field)
documents = [doc["content"] for doc in collection.find()]


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Chunk and Preprocess the data
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs = splitter.create_documents(documents)


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Generate embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
from langchain.vectorstores import FAISS

# Checkpoint for Vector Store
vectorstore = FAISS.from_documents(docs, embedding_model)
vectorstore.save_local("faiss_index")


In [ ]:
# Load saved Vector Store
vectorstore = FAISS.load_local("faiss_index", embedding_model)